<style>
    @media print{
        body {
            position:relative !important;
        }
        .celltag_new_page {
            page-break-before: always !important;
        }
    }
</style>
<div hidden>
    $$
    \newcommand{\reals}{\mathbb{R}}
    \newcommand{\naturals}{\mathbb{N}}
    \newcommand{\integers}{\mathbb{Z}}
    \newcommand{\prob}{\mathbb{P}}
    \newcommand{\expect}{\mathbb{E}}
    \newcommand{\b}[1]{\mathbf{#1}}
    \newcommand{\bs}[1]{\boldsymbol{#1}}
    \newcommand{\c}[1]{\mathcal{#1}}
    \newcommand{mat}[2]{\left[\,\begin{array}{#1}#2\end{array}\,\right]}
    $$
</div>

# COMPSCI 371 Homework 8

_**Group Members:**_

### Problem 0 (3 points)

## Part 1: MLP Back-Propagation: Mathematics

### Problem 1.1 (Exam Style)

### Problem 1.2 (Exam Style)

### Problem 1.3 (Exam Style)

### Problem 1.4 (Exam Style)

\begin{eqnarray*}
\b{g}_{V_2} &=& \b{g}_{\b{z}} \cdot J_{V_2} \\
\b{g}_{\b{b}_2} &=& \\
\b{g}_{\b{q}} &=& 
\end{eqnarray*}

### Problem 1.5 (Exam Style)

Forward pass:

\begin{eqnarray*}
\b{x} &=&  \\
\b{p} &=&  \\
\b{q} &=&  \\
\b{z} &=&  \\
\lambda &=& 
\end{eqnarray*}

Local Jacobians:

\begin{eqnarray*}
J_{V_2} &=&  \\
J_{\b{b}_2} &=&  \\
J_{\b{q}} &=&  \\
J_{\b{p}} &=&  \\
J_{V_1} &=&  \\
J_{\b{b}_1} &=& 
\end{eqnarray*}

Gradients:

\begin{eqnarray*}
\b{g}_{\b{z}} &=&  \\
\b{g}_{V_2} &=&  \\
\b{g}_{\b{b}_2} &=&  \\
\b{g}_{\b{q}} &=&  \\
\b{g}_{\b{p}} &=&  \\
\b{g}_{V_1} &=&  \\
\b{g}_{\b{b}_1} &=& 
\end{eqnarray*}

## Part 2: MLP Back-Propagation: Code 

In [1]:
!pip install numpy


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [89]:
import numpy as np
from types import SimpleNamespace

tests = [
    SimpleNamespace(
        v1=np.array(((2, -1), (0, 3), (2, 1)), dtype=float),
        b1=np.array((2, -1, 3), dtype=float),
        v2=np.array(((2, 1, -3), (3, -1, 2)), dtype=float),
        b2=np.array((4, 2), dtype=float),
        x=np.array((-1, 2), dtype=float),
        y=np.array((-4, 1), dtype=float),
        loss='quadratic'
    ),
    SimpleNamespace(
        v1=np.array(((2, -1, 0), (0, 1, 3)), dtype=float),
        b1=np.array((2, -1), dtype=float),
        v2=np.array((2, -3), dtype=float),
        b2=np.array(4, dtype=float),
        x=np.array((-1, 2, 0), dtype=float),
        y=np.array(3, dtype=float),
        loss='quadratic'
    ),
    SimpleNamespace(
        v1=np.array(((1, 2), (1, 0)), dtype=float),
        b1=np.array((-5, 2), dtype=float),
        v2=np.array(((2, -3), (1, 0), (-1, -1)), dtype=float),
        b2=np.array((1, 3, -2), dtype=float),
        x=np.array((-1, 2), dtype=float),
        y=np.array(2, dtype=int),
        loss='hinge'
    )
]

### Problem 2.1

In [90]:
def array_string(a):
    if np.isscalar(a) or a.ndim == 0:
        return '{:.4g}'.format(a)
    return '(' + ', '.join([array_string(x) for x in a]) + ')'

In [91]:
def dot(a, b):
    return a * b if np.isscalar(a) or np.isscalar(b)\
        else np.einsum('...i,i...', a, b)

In [92]:
def fmt_array(d):
    if np.isscalar(d) or d.ndim == 0:
        if float(d).is_integer():
            return str(int(d))
        else:
            return '{:.4g}'.format(d)
    return '(' + ', '.join([fmt_array(r) for r in d]) + ')'

In [93]:

def multi_dot(x, y):
    return x * y if np.isscalar(x) or np.isscalar(y) else np.einsum('...i,i...', x, y)

In [94]:

def init_affine(M, c):
    return {
        'layer_type': 'affine',
        'weights': M,
        'bias': c,
        'store_input': True,
        'input_val': None,
        'grad_weights': None,
        'grad_bias': None
    }

In [95]:

def affine_forward(lay, val):
    if lay['store_input']:
        lay['input_val'] = val.copy()
    return np.dot(lay['weights'], val) + lay['bias']

In [119]:
def affine_backward(lay, incoming_grad):
    inp = lay['input_val']
    

    if np.isscalar(incoming_grad):
        incoming_grad = np.array([incoming_grad])
    if np.isscalar(inp):
        inp = np.array([inp])
  
    lay['grad_weights'] = np.outer(incoming_grad, inp) if lay['weights'].ndim > 1 else incoming_grad * inp
    lay['grad_bias'] = incoming_grad
 
    if lay['weights'].ndim == 1:
        return lay['weights'] * incoming_grad
    else:
        return np.dot(lay['weights'].T, incoming_grad)


In [120]:

def init_relu():
    return {
        'layer_type': 'relu',
        'store_input': True,
        'input_val': None
    }

In [121]:
def relu_forward(lay, val):
    if lay['store_input']:
        lay['input_val'] = val.copy()
    return np.maximum(0, val)

In [122]:

def relu_backward(lay, incoming_grad):
    inp = lay['input_val']
    return incoming_grad * (inp > 0)

In [123]:
def quad_loss_forward(tgt, pred):
    diff = pred - tgt
    return 0.5 * np.dot(diff, diff)

In [124]:
def quad_loss_backward(tgt, pred):
    return (pred - tgt)

In [125]:
def hinge_loss_forward(tgt, pred):
    s = np.log(np.sum(np.exp(pred)))
    return s - pred[int(tgt)]

In [126]:
def hinge_loss_backward(tgt, pred):
    exp_preds = np.exp(pred)
    sm = exp_preds / np.sum(exp_preds)
    grads = sm.copy()
    grads[int(tgt)] -= 1.0
    return grads

In [127]:
def create_network(layers, loss_type, decision_func=None):
    return {
        'layers': layers,
        'loss_type': loss_type,
        'decision_func': decision_func,
        'store_input': True,
        'loss_value': None,
        'net_input': None,
        'prediction': None,
        'layer_grads': []
    }

In [128]:
def switch_to_inference(net):
    net['store_input'] = False
    for lay in net['layers']:
        lay['store_input'] = False
        lay['input_val'] = None

In [129]:
def switch_to_training(net):
    net['store_input'] = True
    for lay in net['layers']:
        lay['store_input'] = True

In [130]:
def forward_layer(lay, val):
    if lay['layer_type'] == 'affine':
        return affine_forward(lay, val)
    elif lay['layer_type'] == 'relu':
        return relu_forward(lay, val)
    else:
        raise ValueError("Unrecognized layer type")

In [131]:

def backward_layer(lay, grad_val):
    if lay['layer_type'] == 'affine':
        return affine_backward(lay, grad_val)
    elif lay['layer_type'] == 'relu':
        return relu_backward(lay, grad_val)
    else:
        raise ValueError("Unrecognized layer type")


In [132]:
def forward_pass(net, val, target=None):
    net['net_input'] = val
    if target is None:
        switch_to_inference(net)
        activation_res = val
        for lay in net['layers']:
            activation_res = forward_layer(lay, activation_res)
        if net['decision_func'] is not None:
            return net['decision_func'](activation_res)
        else:
            return activation_res
    else:
        switch_to_training(net)
        activation_res = val
        for lay in net['layers']:
            activation_res = forward_layer(lay, activation_res)
        net['prediction'] = activation_res
        if net['loss_type'] == 'quadratic':
            net['loss_value'] = quad_loss_forward(target, activation_res)
        else:
            net['loss_value'] = hinge_loss_forward(target, activation_res)
        return net['loss_value']

In [133]:
def backward_pass(net, target):
    if net['loss_type'] == 'quadratic':
        init_grad = quad_loss_backward(target, net['prediction'])
    else:
        init_grad = hinge_loss_backward(target, net['prediction'])
    for lay in reversed(net['layers']):
        init_grad = backward_layer(lay, init_grad)

In [134]:
def gather_gradients(net):
    grads = []
    for lay in net['layers']:
        if lay['layer_type'] == 'affine':
            grads.append((lay['grad_weights'], lay['grad_bias']))
        else:
            grads.append(None)
    return grads

In [ ]:
def initialize_layers(test_case):
   
    if isinstance(test_case.b2, np.ndarray):
        return [
            init_affine(test_case.v1, test_case.b1),
            init_relu(),
            init_affine(test_case.v2, test_case.b2)
        ]
    else:
        # Ensure b2 is a 1D array for consistency
        return [
            init_affine(test_case.v1, test_case.b1),
            init_relu(),
            init_affine(test_case.v2.reshape((1, -1)), np.array([test_case.b2]))
        ]

In [ ]:
def setup_network(layer_defs, loss_type):
    """Set up the network with the specified layers and loss function."""
    loss_function = 'quadratic' if loss_type == 'quadratic' else 'hinge'
    decision_func = None if loss_type == 'quadratic' else np.argmax
    return create_network(layer_defs, loss_function, decision_func=decision_func)

In [ ]:
def compute_forward_pass(network, layer_defs, x_input):
    """Compute the forward pass values and store them in a dictionary."""
    forward_vals = {
        'x_inp': x_input,
        'layer1_out': affine_forward(layer_defs[0], x_input),
        'layer2_out': relu_forward(layer_defs[1], affine_forward(layer_defs[0], x_input)),
        'final_out': affine_forward(layer_defs[2], relu_forward(layer_defs[1], affine_forward(layer_defs[0], x_input))),
        'loss': network['loss_value']
    }
    return forward_vals

In [ ]:
def calculate_local_jacobians(layer_defs, forward_vals):
    jacobian_V2_shape = (
        layer_defs[2]['weights'].shape[0], 
        layer_defs[2]['weights'].shape[0], 
        layer_defs[2]['weights'].shape[1] if layer_defs[2]['weights'].ndim > 1 else 1
    )
    jacobian_V1_shape = (
        layer_defs[0]['weights'].shape[0], 
        layer_defs[0]['weights'].shape[0], 
        layer_defs[0]['weights'].shape[1] if layer_defs[0]['weights'].ndim > 1 else 1
    )
    
    bias_b2_shape = layer_defs[2]['bias'].shape[0] if layer_defs[2]['bias'].ndim > 0 else 1
    bias_b1_shape = layer_defs[0]['bias'].shape[0] if layer_defs[0]['bias'].ndim > 0 else 1
    
    local_jacobs = {
        'jacobian_V2': np.zeros(jacobian_V2_shape),
        'jacobian_b2': np.eye(bias_b2_shape),
        'jacobian_layer2': layer_defs[2]['weights'],
        'jacobian_layer1_out': np.diag((forward_vals['layer1_out'] > 0).astype(int)),
        'jacobian_V1': np.zeros(jacobian_V1_shape),
        'jacobian_b1': np.eye(bias_b1_shape)
    }
    return local_jacobs

In [ ]:
def calculate_gradients(network, test_case, layer_defs):
    if network['loss_type'] == 'quadratic':
        grad_final_out = quad_loss_backward(test_case.y, network['prediction'])
    else:
        grad_final_out = hinge_loss_backward(test_case.y, network['prediction'])
    
    grad_layer2_inp = affine_backward(layer_defs[2], grad_final_out)
    grad_layer1_out = relu_backward(layer_defs[1], grad_layer2_inp)
    x_gradient = affine_backward(layer_defs[0], grad_layer1_out)
    
    gradients = gather_gradients(network)
    w_grad1, b_grad1 = gradients[0] if gradients[0] is not None else (None, None)
    w_grad2, b_grad2 = gradients[2] if gradients[2] is not None else (None, None)
    
    grad_vals = {
        'grad_final_out': grad_final_out,
        'grad_w2': w_grad2,
        'grad_b2': b_grad2,
        'grad_layer2_inp': grad_layer2_inp,
        'grad_layer1_out': grad_layer1_out,
        'grad_w1': w_grad1,
        'grad_b1': b_grad1
    }
    return grad_vals

In [141]:
def execute_test(test_case, net_idx):
    layer_defs = initialize_layers(test_case)
    network = setup_network(layer_defs, test_case.loss)
    
    net_loss = forward_pass(network, test_case.x, test_case.y)
    forward_vals = compute_forward_pass(network, layer_defs, test_case.x)
    

    local_jacobs = calculate_local_jacobians(layer_defs, forward_vals)
    
    # Backward pass and gradient calculation
    backward_pass(network, test_case.y)
    grad_vals = calculate_gradients(network, test_case, layer_defs)
    
    return forward_vals, local_jacobs, grad_vals


In [142]:

def show_test_results(net_index, forward_data, jacobians, gradients):

    print(f"Network {net_index}")
    print("Forward Pass:")
    print(f"    Input x: {fmt_array(forward_data['x_inp'])}")
    print(f"    Layer 1 Output (p): {fmt_array(forward_data['layer1_out'])}")
    print(f"    Layer 2 Output (q): {fmt_array(forward_data['layer2_out'])}")
    print(f"    Final Output (z): {fmt_array(forward_data['final_out'])}")
    print(f"    Loss (lambda): {fmt_array(forward_data['loss'])}")

    print("Local Jacobians:")
    print(f"    J_V2: {fmt_array(jacobians['jacobian_V2'])}")
    print(f"    J_b2: {fmt_array(jacobians['jacobian_b2'])}")
    print(f"    J_q: {fmt_array(jacobians['jacobian_layer2'])}")
    print(f"    J_p: {fmt_array(jacobians['jacobian_layer1_out'])}")
    print(f"    J_V1: {fmt_array(jacobians['jacobian_V1'])}")
    print(f"    J_b1: {fmt_array(jacobians['jacobian_b1'])}")
    
    print("Gradients:")
    print(f"    g_z: {fmt_array(gradients['grad_final_out'])}")
    print(f"    g_V2: {fmt_array(gradients['grad_w2'])}")
    print(f"    g_b2: {fmt_array(gradients['grad_b2'])}")
    print(f"    g_q: {fmt_array(gradients['grad_layer2_inp'])}")
    print(f"    g_p: {fmt_array(gradients['grad_layer1_out'])}")
    print(f"    g_V1: {fmt_array(gradients['grad_w1'])}")
    print(f"    g_b1: {fmt_array(gradients['grad_b1'])}")

In [143]:
for index, case in enumerate(test_cases):
    forward_data, local_jacobians, gradients = execute_test(case, index)
    show_test_results(index, forward_data, local_jacobians, gradients)
    print()

Network 0
Forward Pass:
    Input x: (-1, 2)
    Layer 1 Output (p): (-2, 5, 3)
    Layer 2 Output (q): (0, 5, 3)
    Final Output (z): (0, 3)
    Loss (lambda): 10
Local Jacobians:
    J_V2: (((0, 0, 0), (0, 0, 0)), ((0, 0, 0), (0, 0, 0)))
    J_b2: ((1, 0), (0, 1))
    J_q: ((2, 1, -3), (3, -1, 2))
    J_p: ((0, 0, 0), (0, 1, 0), (0, 0, 1))
    J_V1: (((0, 0), (0, 0), (0, 0)), ((0, 0), (0, 0), (0, 0)), ((0, 0), (0, 0), (0, 0)))
    J_b1: ((1, 0, 0), (0, 1, 0), (0, 0, 1))
Gradients:
    g_z: (4, 2)
    g_V2: ((0, 20, 12), (0, 10, 6))
    g_b2: (4, 2)
    g_q: (14, 2, -8)
    g_p: (0, 2, -8)
    g_V1: ((0, 0), (-2, 4), (8, -16))
    g_b1: (0, 2, -8)

Network 1
Forward Pass:
    Input x: (-1, 2, 0)
    Layer 1 Output (p): (-2, 1)
    Layer 2 Output (q): (0, 1)
    Final Output (z): 1
    Loss (lambda): 2
Local Jacobians:
    J_V2: (((0), (0)), ((0), (0)))
    J_b2: ((1))
    J_q: (2, -3)
    J_p: ((0, 0), (0, 1))
    J_V1: (((0, 0, 0), (0, 0, 0)), ((0, 0, 0), (0, 0, 0)))
    J_b1: ((1, 